In [2]:
import boto3
from boto3.dynamodb.conditions import Attr
from datetime import datetime
from decimal import Decimal
import csv
import pprint

In [4]:
#conectando ao dynamo
dynamodb = boto3.resource('dynamodb',
                          region_name='us-east-1',
                          aws_access_key_id='#######',
                          aws_secret_access_key='######')

In [5]:
#tabela para pegar os dados do serasa
table = dynamodb.Table('evaluation-report-data-prod')

#tabela para pegar os dados do boa vista e quod
table2 = dynamodb.Table('experimento-comparacao-prod')

#tabela para pegar os dados do boa vista e quod - banco teste
table3 = dynamodb.Table('experimento-comparacao')

In [6]:
cpfs = []
f = Attr('bureau').eq('boa_vista')
response = table2.scan(FilterExpression=f)
while len(response['Items']) > 0:
    for i in response['Items']:
        cpfs.append(i['document'])
    if 'LastEvaluatedKey' in response:
        response = table2.scan(FilterExpression=f,ExclusiveStartKey=response['LastEvaluatedKey'])
    else:
        break

In [8]:
relatorio = {}
for cpf in cpfs:
    response = table2.get_item(Key={'document': cpf, 'bureau': 'serasa'})
    if 'Item' in response:
        i = response['Item']
        relatorio[cpf] = {}
        relatorio[cpf]['serasa'] = {}
        relatorio[cpf]['serasa']['score'] = i['data']['laudo']['score_serasa']
        relatorio[cpf]['serasa']['consultas anteriores'] = i['data']['laudo']['rede_ficha']['pesquisas_anteriores_api']['result']['total']                
        relatorio[cpf]['serasa']['pendencias financeiras'] = len(i['data']['laudo']['restricoes_financeiras']['result']['pendencias']['detalhes'])
        relatorio[cpf]['serasa']['protestos'] = len(i['data']['laudo']['restricoes_financeiras']['result']['protestos']['detalhes'])
        relatorio[cpf]['serasa']['cheques sem fundo'] = len(i['data']['laudo']['restricoes_financeiras']['result']['cheques']['detalhes'])

        relatorio[cpf]['boa vista'] = {}
        relatorio[cpf]['boa vista']['score'] = 0
        relatorio[cpf]['boa vista']['consultas anteriores'] = 0
        relatorio[cpf]['boa vista']['pendencias financeiras'] = 0
        relatorio[cpf]['boa vista']['protestos'] = 0
        relatorio[cpf]['boa vista']['cheques sem fundo'] = 0

        relatorio[cpf]['quod'] = {}
        relatorio[cpf]['quod']['score'] = 0
        relatorio[cpf]['quod']['consultas anteriores'] = 0
        relatorio[cpf]['quod']['pendencias financeiras'] = 0
        relatorio[cpf]['quod']['protestos'] = 0
        relatorio[cpf]['quod']['cheques sem fundo'] = 0

        print('CPF ADICIONADO NA LISTA: ', cpf)
        print('Total de CPFs: ', len(relatorio))
        print('----')

CPF ADICIONADO NA LISTA:  02118484097
Total de CPFs:  1
----
CPF ADICIONADO NA LISTA:  48197617813
Total de CPFs:  2
----
CPF ADICIONADO NA LISTA:  04784462848
Total de CPFs:  3
----
CPF ADICIONADO NA LISTA:  02971922995
Total de CPFs:  4
----
CPF ADICIONADO NA LISTA:  84758422087
Total de CPFs:  5
----
CPF ADICIONADO NA LISTA:  32698119691
Total de CPFs:  6
----
CPF ADICIONADO NA LISTA:  61294697307
Total de CPFs:  7
----
CPF ADICIONADO NA LISTA:  68564783991
Total de CPFs:  8
----
CPF ADICIONADO NA LISTA:  02900983940
Total de CPFs:  9
----
CPF ADICIONADO NA LISTA:  85798594505
Total de CPFs:  10
----
CPF ADICIONADO NA LISTA:  22865708845
Total de CPFs:  11
----
CPF ADICIONADO NA LISTA:  41815696915
Total de CPFs:  12
----
CPF ADICIONADO NA LISTA:  49182340030
Total de CPFs:  13
----
CPF ADICIONADO NA LISTA:  06686601985
Total de CPFs:  14
----
CPF ADICIONADO NA LISTA:  08573287942
Total de CPFs:  15
----
CPF ADICIONADO NA LISTA:  59161531987
Total de CPFs:  16
----
CPF ADICIONADO NA

In [9]:
#relatorio apos a etapa 1 - pegando os dados do serasa em nossa base 
pp = pprint.PrettyPrinter(indent=5)
pp.pprint(relatorio)

{    '01110481098': {    'boa vista': {    'cheques sem fundo': 0,
                                           'consultas anteriores': 0,
                                           'pendencias financeiras': 0,
                                           'protestos': 0,
                                           'score': 0},
                         'quod': {    'cheques sem fundo': 0,
                                      'consultas anteriores': 0,
                                      'pendencias financeiras': 0,
                                      'protestos': 0,
                                      'score': 0},
                         'serasa': {    'cheques sem fundo': 0,
                                        'consultas anteriores': Decimal('3'),
                                        'pendencias financeiras': 0,
                                        'protestos': 0,
                                        'score': Decimal('569')}},
     '01329990641': {    'boa vista': {    

In [9]:
#pegando os dados da boa vista

for cpf in relatorio:
    response = table2.get_item(Key={'document': cpf, 'bureau': 'boa_vista'})
    if 'Item' in response: 
        #pegando score
        corte_um = response['Item']['data'].split('<SPCA-601-SCORE>')
        corte_dois = corte_um[1].split('</SPCA-601-SCORE>')
        score_boa_vista = corte_dois[0]
        relatorio[cpf]['boa vista']['score'] = int(score_boa_vista)

        #pegando consultas anteriores
        if '<SPCA-111-TOTAL>' in response['Item']['data']:
            corte_um = str(response['Item']['data']).split('<SPCA-111-TOTAL>')
            corte_dois = corte_um[1].split('</SPCA-111-TOTAL>')
            consultas_anteriores_boa_vista = corte_dois[0]
            relatorio[cpf]['boa vista']['consultas anteriores'] = consultas_anteriores_boa_vista

        #pegando pendencias financeiras parte 1 (devedor)
        pendencias_financeiras_boa_vista = 0
        if '<SPCA-108-DEVEDOR>' in response['Item']['data']:
            corte_um = str(response['Item']['data']).split('<SPCA-108-DEVEDOR>')
            corte_dois = corte_um[1].split('</SPCA-108-DEVEDOR>')
            pendencias_financeiras_boa_vista = corte_dois[0]
            relatorio[cpf]['boa vista']['pendencias financeiras'] = pendencias_financeiras_boa_vista

        #pegando pendencias financeiras parte 2 (avalista)
        if '<SPCA-108-AVALISTA>' in response['Item']['data']:
            corte_um = str(response['Item']['data']).split('<SPCA-108-AVALISTA>')
            corte_dois = corte_um[1].split('</SPCA-108-AVALISTA>')
            pendencias_financeiras_boa_vista = corte_dois[0]
            #somando devedor com avalista para termos o total
            relatorio[cpf]['boa vista']['pendencias financeiras'] = relatorio[cpf]['boa vista']['pendencias financeiras'] + int(pendencias_financeiras_boa_vista)

        #pegando protestos
        if '<SPCA-022-PROTESTO>' in response['Item']['data']:
            corte_um = str(response['Item']['data']).split('<SPCA-109-PROTESTO>')
            corte_dois = corte_um[1].split('</SPCA-109-PROTESTO>')
            protestos_boa_vista = corte_dois[0]
            relatorio[cpf]['boa vista']['protestos'] = protestos_boa_vista

        #pegando cheques sem fundo 
        relatorio[cpf]['boa vista']['cheques sem fundo'] = 0
        if '<SPCA-214-TOTAL-12>' in response['Item']['data']:
            print(cpf)
            corte_um = str(response['Item']['data']).split('<SPCA-214-TOTAL-12>')
            corte_dois = corte_um[1].split('</SPCA-214-TOTAL-12>')
            cheques_sem_fundo = corte_dois[0]
            relatorio[cpf]['boa vista']['cheques sem fundo'] = cheques_sem_fundo

        if '<SPCA-214-TOTAL-13>' in response['Item']['data']:
            corte_um = str(response['Item']['data']).split('<SPCA-214-TOTAL-13>')
            corte_dois = corte_um[1].split('</SPCA-214-TOTAL-13>')
            cheques_sem_fundo = corte_dois[0]
            relatorio[cpf]['boa vista']['cheques sem fundo'] = relatorio[cpf]['boa vista']['cheques sem fundo'] + cheques_sem_fundo

        if '<SPCA-214-TOTAL-14>' in response['Item']['data']:
            corte_um = str(response['Item']['data']).split('<SPCA-214-TOTAL-14>')
            corte_dois = corte_um[1].split('</SPCA-214-TOTAL-14>')
            cheques_sem_fundo = corte_dois[0]
            relatorio[cpf]['boa vista']['cheques sem fundo'] = relatorio[cpf]['boa vista']['cheques sem fundo'] + cheques_sem_fundo

14525655704


In [41]:
#relatorio apos a etapa 2 - pegando os dados do boa vista
pp = pprint.PrettyPrinter(indent=5)
pp.pprint(relatorio)

{    '01110481098': {    'boa vista': {    'cheques sem fundo': 0,
                                           'consultas anteriores': '1',
                                           'pendencias financeiras': 0,
                                           'protestos': 0,
                                           'score': 862},
                         'quod': {    'cheques sem fundo': 0,
                                      'consultas anteriores': 0,
                                      'pendencias financeiras': 0,
                                      'protestos': 0,
                                      'score': 0},
                         'serasa': {    'cheques sem fundo': 0,
                                        'consultas anteriores': Decimal('3'),
                                        'pendencias financeiras': 0,
                                        'protestos': 0,
                                        'score': Decimal('569')}},
     '01329990641': {    'boa vista': {

In [42]:
#pegando os dados do QUOD

for cpf in relatorio:
    response = table2.get_item(Key={'document': cpf, 'bureau': 'quod'})
    if 'Records' in response['Item']['data']['QuodReportResponseEx']['Response']:
        dados = response['Item']['data']['QuodReportResponseEx']['Response']['Records']['Record'][0]
        #pegando score
        if 'QuodScore' in dados:
            relatorio[cpf]['quod']['score'] = int(dados['QuodScore']['Score'])
        #pegando consultas anteriores
        if 'Inquiries' in dados:
            relatorio[cpf]['quod']['consultas anteriores'] = dados['Inquiries']['InquiryCountLast30Days']+dados['Inquiries']['InquiryCountLast31to60Days']+dados['Inquiries']['InquiryCountLast61to90Days']+dados['Inquiries']['InquiryCountMore90Days']
        if 'Negative' in dados:
            if 'CcfApontamentos' in dados['Negative']:
                if 'CcfApontamento' in dados['Negative']['CcfApontamentos']:
                    #pegando cheques sem fundo
                    relatorio[cpf]['quod']['cheques sem fundo'] = len(dados['Negative']['CcfApontamentos']['CcfApontamento'])
            if 'Apontamentos' in dados['Negative']:
                if 'Apontamento' in dados['Negative']['Apontamentos']:
                    #pegando pendências financeiras
                    relatorio[cpf]['quod']['pendencias financeiras'] = len(dados['Negative']['Apontamentos']['Apontamento'])

In [43]:
#relatorio apos a etapa 3 - pegando os dados do quod
pp = pprint.PrettyPrinter(indent=5)
pp.pprint(relatorio)

{    '01110481098': {    'boa vista': {    'cheques sem fundo': 0,
                                           'consultas anteriores': '1',
                                           'pendencias financeiras': 0,
                                           'protestos': 0,
                                           'score': 862},
                         'quod': {    'cheques sem fundo': 0,
                                      'consultas anteriores': Decimal('1'),
                                      'pendencias financeiras': 0,
                                      'protestos': 0,
                                      'score': 765},
                         'serasa': {    'cheques sem fundo': 0,
                                        'consultas anteriores': Decimal('3'),
                                        'pendencias financeiras': 0,
                                        'protestos': 0,
                                        'score': Decimal('569')}},
     '01329990641': {    '

In [44]:
#gerando csv a partir do relatorio dict
relatorio1 = relatorio
csv_columns = [
                'CPF', 'SCORE SERASA', 'SCORE BOA VISTA', 'SCORE QUOD',
                'SCORE SERASA-BOAVISTA', 'SCORE SERASA-QUOD', 'MENOR DIFERENÇA', 'SCORE BOA VISTA-QUOD',
                'SERASA CHEQUES SEM FUNDO', 'BOA VISTA CHEQUES SEM FUNDO', 'QUOD CHEQUES SEM FUNDO',
                'SERASA CONSULTAS ANTERIORES', 'BOA VISTA CONSULTAS ANTERIORES', 'QUOD CONSULTAS ANTERIORES',
                'SERASA PENDENCIAS FINANCEIRAS', 'BOA VISTA PENDENCIAS FINANCEIRAS', 'QUOD PENDENCIAS FINANCEIRAS',
                'SERASA PROTESTOS', 'BOA VISTA PROTESTOS', 'QUOD PROTESTOS'
              ]
with open('comparacao_apis.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=';')
    spamwriter.writerow(csv_columns)
    for key in relatorio1:
        listaRow = list()
        listaRow.append(key)
        listaRow.append(relatorio[key]['serasa']['score'])
        listaRow.append(relatorio[key]['boa vista']['score'])
        listaRow.append(relatorio[key]['quod']['score']) 
        
        diff_serasa_boa_vista = abs(int(relatorio[key]['serasa']['score'])-int(relatorio[key]['boa vista']['score']))
        diff_serasa_quod = abs(int(relatorio[key]['serasa']['score'])-int(relatorio[key]['quod']['score']))
        diff_boa_vista_quod = abs(int(relatorio[key]['boa vista']['score'])-int(relatorio[key]['quod']['score']))

        listaRow.append(diff_serasa_boa_vista)
        listaRow.append(diff_serasa_quod)
        
        if diff_serasa_boa_vista > diff_serasa_quod:
            minor_diff = 'QUOD'
        elif diff_serasa_boa_vista < diff_serasa_quod:
            minor_diff = 'BOA VISTA'
        elif diff_serasa_boa_vista == diff_serasa_quod:
            minor_diff = '='
            
        listaRow.append(minor_diff)
        listaRow.append(diff_boa_vista_quod)
        
        listaRow.append(relatorio[key]['serasa']['cheques sem fundo'])
        listaRow.append(relatorio[key]['boa vista']['cheques sem fundo'])
        listaRow.append(relatorio[key]['quod']['cheques sem fundo'])
        listaRow.append(relatorio[key]['serasa']['consultas anteriores'])
        listaRow.append(relatorio[key]['boa vista']['consultas anteriores'])
        listaRow.append(relatorio[key]['quod']['consultas anteriores'])
        listaRow.append(relatorio[key]['serasa']['pendencias financeiras'])
        listaRow.append(relatorio[key]['boa vista']['pendencias financeiras'])
        listaRow.append(relatorio[key]['quod']['pendencias financeiras'])
        listaRow.append(relatorio[key]['serasa']['protestos'])
        listaRow.append(relatorio[key]['boa vista']['protestos'])
        listaRow.append(relatorio[key]['quod']['protestos'])
        spamwriter.writerow(listaRow)
print('Script executado com sucesso!')

Script executado com sucesso!
